## Initial configuration

#### To start working with this particular notebook, you need to provide necessary credential and settings
#### Below is an template of configuration, which is necessary prepare aside of this notebook and copy & paste all content in triple quotes to the next cell's input field

    """
    {
    "CLOUDANT_API_KEY": "xxx",
    "CLOUDANT_URL": "xxx",
    "UTILS_BUCKET": "notebook-utils-bucket",
    "HEIGHTS_TIFF_FILENAME": "WSF3Dv3_Kenya.tif",
    "DB_NAME": "xxx",
    "COS_ENDPOINT_URL": "xxx",
    "COS_APIKEY": "xxx",
    "TYPE_SOURCE_FILTER": "area",
    "AREA_TRESHOLD": 0
    }
    """

In [ ]:
# Set config
import getpass
import json


# read config
config_str = getpass.getpass('Enter your prepared config: ')
config = json.loads(config_str)

In [ ]:
# !pip install geopandas
# !pip install shapely
# !pip install ibmcloudant
# !pip install urllib3==1.26.16
# !pip install requests==2.25.0

In [4]:
import argparse
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
from tensorflow import keras
# import keras_tuner as kt
import fnmatch
import cv2
import gc
# from ttictoc import tic, toc
import csv
import random
import datetime
from sklearn.utils import class_weight

import keras
from keras.models import Sequential
from tensorflow.keras import layers, models
from keras import backend as B
from tensorflow.keras.applications import DenseNet121, EfficientNetV2B3
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, LambdaCallback, Callback
from tensorflow.keras.optimizers import SGD, RMSprop, Adam, Adadelta, Adagrad

from keras.layers import Input, Concatenate, Conv2D, Dense, BatchNormalization
from keras.models import Model

from keras.layers import BatchNormalization
import sklearn.preprocessing as SKP
import numpy as np
import shutil
import io
import ibm_boto3
from botocore.client import Config
import tarfile
import base64
from tqdm import tqdm
import time
import pandas as pd
from collections import Counter
from scipy import stats
import scipy as SCP
import psutil
import os
import zipfile
import geopandas as gpd
import shapely
# from ibmcloudant.cloudant_v1 import CloudantV1
# from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
import cv2

# from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

In [6]:
architecture = 'DenseNet121_L1_4INP'
configuration = 'CFG002'


cos_client = ibm_boto3.resource(service_name='s3',
    ibm_api_key_id=config["COS_APIKEY"],
    ibm_service_instance_id=config["ML_MODELS_BUCKET_CRN"],
    ibm_auth_endpoint='https://iam.bluemix.net/oidc/token',
    config=Config(signature_version='oauth'),
    endpoint_url=config["COS_ENDPOINT_URL"]
)


In [21]:
l1ml_datasets_bucket = cos_client.Bucket('l1ml-datasets')

dataset_name = 'OSM_VIDA_derivative+overture_merged_L1_SMOD_heights_images.parquet'
l1ml_datasets_bucket.download_file(dataset_name, dataset_name)

l1ml_datasets_bucket.download_file('regions_to_remove.parquet', 'regions_to_remove.parquet')



In [22]:
trm_df = gpd.read_parquet('regions_to_remove.parquet')
trm_df

,geometry,name
0,"POLYGON ((34.77008 -0.10944, 34.75269 -0.12266...",Nyalenda
0,"POLYGON ((36.83574 -1.3087, 36.8368 -1.30969, ...",Mukuru_1
0,"POLYGON ((36.86267 -1.31371, 36.86347 -1.31469...",Mukuru_2
0,"POLYGON ((36.79552 -1.31941, 36.7982 -1.319, 3...",Kibera


In [23]:
ML_df = pd.read_parquet(dataset_name)


ML_df = ML_df[(ML_df.use_for_training == 'Yes') & (ML_df.vida_confidence >= 0.9) & ~ML_df.image_source_bytes.isnull()]

ML_df[['image_ML_type', 'L1_class']].groupby(['image_ML_type', 'L1_class']).agg({'L1_class': ['count']})

L1_class
                                count
image_ML_type L1_class               
test          nonresidential     4018
              residential        6405
train         nonresidential    28100
              residential       44836
validation    nonresidential     8031
              residential       12810

In [24]:
res_df = ML_df[ML_df.L1_class == 'residential']
nonres_df = ML_df[ML_df.L1_class == 'nonresidential']

In [ ]:
nonres_df = nonres_df[~nonres_df.building_tag.isin(['construction', 'foundation'])]
nonres_df

In [26]:
trm_dfs = []

for trm_row in trm_df.itertuples():
    
    min_lon, min_lat, max_lon, max_lat = trm_row.geometry.bounds

    # fetch entries in district boundaries
    df = nonres_df[
                (nonres_df.area_in_meters <= 300) & \
                (nonres_df.latitude >= min_lat) & \
                (nonres_df.latitude <= max_lat) & \
                (nonres_df.longitude >= min_lon) & \
                (nonres_df.longitude <= max_lon)
            ].copy()

    # keep only buildings inside district polygon
    df['buildings_in_polygon'] = [trm_row.geometry.contains(shapely.Point(row.longitude, row.latitude)) for row in df.itertuples()]
    df = df[df.buildings_in_polygon == True]
    df = df.drop(['buildings_in_polygon'], axis=1)
    
    trm_dfs.append(df)



In [ ]:
trm_df_buildings = pd.concat(trm_dfs)
trm_df_buildings

In [ ]:
Counter(trm_df_buildings.building_tag)

In [29]:
trm_df_buildings_ids = list(trm_df_buildings.id)

In [ ]:
nonres_df = nonres_df[~nonres_df.id.isin(trm_df_buildings_ids)]
nonres_df

In [31]:
ML_df = pd.concat([res_df, nonres_df])

In [ ]:
ML_df

In [ ]:
def assign_label(idx):
#     parts = [0, int(data_len * 0.8), int(data_len * 0.9), data_len]
    
    if str(idx)[-1] in ['0', '1', '2', '3', '4', '5', '6']:
        return 'train'
    elif str(idx)[-1] in ['7', '8']:
        return 'validation'
    elif str(idx)[-1] in ['9']:
        return 'test'

data_len = len(ML_df)
ML_df['index_column'] = [i for i in range(len(ML_df))]
ML_df['image_ML_type'] = ["initval" for _ in range(len(ML_df))]

for ml_class in list(set(ML_df['L1_class'])):
    
    
    ML_df = ML_df.sort_values('area_in_meters', ascending=True)
    ml_class_data_idxs = ML_df[ML_df['L1_class'] == ml_class].index.tolist()
    for row_idx, df_idx in enumerate(ml_class_data_idxs):
        
        ML_df.at[df_idx, 'image_ML_type'] = assign_label(row_idx)
        
split_result = ML_df[['image_ML_type', 'L1_class', 'index_column']].groupby(['image_ML_type', 'L1_class']).count()
split_result['split in %'] = round(100 * split_result['index_column'] / data_len, 3)
print(split_result)

In [ ]:
gML = ML_df[['image_ML_type', 'L1_class']].reset_index().groupby(['image_ML_type', 'L1_class'])["L1_class"].count().reset_index(name="count")
gML = pd.DataFrame(gML)
gML


In [ ]:
ML_df[ML_df.L1_class == 'residential'].describe()

In [ ]:
Counter(ML_df[ML_df.L1_class == 'residential'].building_tag)

In [ ]:
Counter(ML_df[ML_df.L1_class == 'nonresidential'].building_tag)

In [49]:
base_path = os.getcwd()
data_path = os.path.join(base_path, 'Sentinel_set/')

models_dir = os.path.join(base_path, 'model_files')
checkpoints_and_metadata = os.path.join(base_path, 'model_checkpoints_and_metadata')

# delete models dir if exists
try:
    shutil.rmtree(models_dir)
    shutil.rmtree(checkpoints_and_metadata)
except:
    pass

# recreste models dir
os.makedirs(models_dir, exist_ok = True)
os.makedirs(checkpoints_and_metadata, exist_ok = True)

In [50]:
print('To ensure that necessary folders were created')
# os.listdir('/home/wsuser/work/')

To ensure that necessary folders were created


In [51]:
from PIL import Image

In [52]:
get_class_number = {
    'nonresidential': 0,
    'residential': 1
}

In [53]:
normalize_area = 20_000
normalize_height = 20
normalize_smod = 6

In [54]:
folders_tree = {
    'train': ['nonresidential', 'residential'],
    'test': ['nonresidential', 'residential'],
    'validation': ['nonresidential', 'residential'],
}

train_images, train_numeric, train_labels = [], [], []
validation_images, validation_numeric, validation_labels = [], [], []
test_images, test_numeric, test_labels = [], [], []


def create_learning_sample(row):
    
    image_data = base64.b64decode(row.image_source_bytes)
    img = Image.open(io.BytesIO(image_data))
    reshaped_img = np.array(img.resize((124, 124), Image.Resampling.NEAREST))
    output = SKP.label_binarize([get_class_number[row.L1_class]], classes=np.arange(2))[0]
    
    return reshaped_img, output

# limit = 10_000

for type_folder, class_folders in folders_tree.items():
    
#     c_counter = {
#         'nonresidential': 0,
#         'residential': 0,
#     }
    
    for classfolder in class_folders:
        
        folder_path = os.path.join(data_path, type_folder, classfolder)
        
        class_images = ML_df[(ML_df.image_ML_type == type_folder) & (ML_df.L1_class == classfolder)]
        
        for img_idx, row in enumerate(class_images.itertuples()):
              
#             if c_counter[row.L1_class] > limit: break
            
            if type_folder == "train":
                
                reshaped_img, output = create_learning_sample(row)
                
                train_images.append(reshaped_img)
                train_numeric.append([row.area_in_meters / normalize_area, row.SMOD_id/normalize_smod])
                train_labels.append(output)
                
            elif type_folder == "validation":
                
                reshaped_img, output = create_learning_sample(row)

                validation_images.append(reshaped_img)
                validation_numeric.append([row.area_in_meters / normalize_area, row.SMOD_id/normalize_smod])
                validation_labels.append(output)
                
            elif type_folder == "test":
                
                reshaped_img, output = create_learning_sample(row)
                
                test_images.append(reshaped_img)
                test_numeric.append([row.area_in_meters / normalize_area, row.SMOD_id/normalize_smod])
                test_labels.append(output)
                
#             c_counter[row.L1_class] += 1

len_train_images = len(train_images)

In [55]:
from keras.preprocessing.image import ImageDataGenerator
batch_size=32
# Define the image transformations here
gen = ImageDataGenerator(
                            horizontal_flip = True, 
                            vertical_flip = True, 
                            rotation_range = 20,
                        )

# Here is the function that merges our two generators
# We use the exact same generator with the same random seed for both the y and angle arrays
def gen_flow_for_two_inputs(X1, X2, y):
    genX1 = gen.flow(X1,y,  batch_size=batch_size,seed=123)
    genX2 = gen.flow(X1,X2, batch_size=batch_size,seed=123)
    
    while True:
        X1i = genX1.next()
        X2i = genX2.next()
        
        #Assert arrays are equal - this was for peace of mind, but slows down training
#         np.testing.assert_array_equal(X1i[0],X2i[0])

        yield [X1i[0], X2i[1]], X1i[1]

# Finally create generator

train_images, train_numeric, train_labels = np.array(train_images), np.array(train_numeric), np.array(train_labels)
validation_images, validation_numeric, validation_labels = np.array(validation_images), np.array(validation_numeric), np.array(validation_labels)

train_generator = gen_flow_for_two_inputs(train_images, train_numeric, train_labels)

In [59]:
del train_images
del train_numeric
del train_labels

collected = gc.collect()
# Prints Garbage collector 
# as 0 object
print("Garbage collector: collected %d objects." % collected)

Garbage collector: collected 184 objects.


In [60]:
# Define two input layers
image_input = Input((124, 124, 3), name="image_input")
numeric_input = Input((2,), name="numeric_input")

# Convolution + Flatten for the image
base_cnn_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(124, 124, 3))
base_cnn_model.trainable = True

base_cnn_model_input = base_cnn_model(image_input)

numeric_input_layer = Dense(4, activation='tanh', use_bias=True)(numeric_input)

GAP2D_layer = GlobalAveragePooling2D()(base_cnn_model_input)

print('GAP2D_layer created')
# Concatenate the convolutional features and the vector input
concat_layer = Concatenate()([numeric_input_layer, GAP2D_layer])

dropout_layer = Dropout(0.4)(concat_layer)
dense_layer = Dense(200, activation='elu', use_bias=True)(dropout_layer)
dropout_layer = Dropout(0.4)(dense_layer)
dense_layer = Dense(100, activation='elu', use_bias=True)(dropout_layer)
dropout_layer = Dropout(0.4)(dense_layer)
dense_layer = Dense(50, activation='elu', use_bias=True)(dropout_layer)
output = Dense(1, activation='sigmoid')(dense_layer)

# define a model with a list of two inputs
model = Model(inputs=[image_input, numeric_input], outputs=output)

model.summary()


29084464/29084464 [==============================] - 1s 0us/step
GAP2D_layer created
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 image_input (InputLayer)       [(None, 124, 124, 3  0           []                               
                                )]                                                                
                                                                                                  
 numeric_input (InputLayer)     [(None, 2)]          0           []                               
                                                                                                  
 densenet121 (Functional)       (None, 3, 3, 1024)   7037504     ['image_input[0][0]']            
                                                                                                  
 dense (D

In [61]:
import itertools
import sklearn.metrics as SKM
import seaborn as sns

file_writer_cm = tf.summary.create_file_writer(checkpoints_and_metadata)
test_inputs = [np.array(test_images), np.array(test_numeric)]
test_labels = np.array(test_labels)

nonres_count = gML[(gML.image_ML_type == 'test') & (gML.L1_class == 'nonresidential')]['count'].iloc[0]
res_count = gML[(gML.image_ML_type == 'test') & (gML.L1_class == 'residential')]['count'].iloc[0]

def plot_to_image(figure):
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    plt.close(figure)
    buf.seek(0)

    digit = tf.image.decode_png(buf.getvalue(), channels=4)
    digit = tf.expand_dims(digit, 0)

    return digit

def plot_confusion_matrix():
    
    print(f"Evaluating model ")
     
    predictions = model.predict(test_inputs, verbose=1, batch_size=len(test_labels))

    y_pred = np.argmax(predictions, axis=1)

    cf_mtx = SKM.confusion_matrix(test_labels, y_pred)
    # group_counts = ["{0:0.0f}".format(value) for value in cf_mtx.flatten()]
    # group_percentages = ["{0:.2%}".format(value) for value in cf_mtx.flatten()/np.sum(cf_mtx)]

    counts = [
        nonres_count, nonres_count,
        res_count, res_count,  
        ]
    
    group_names = [
        'Correctly predicted nonres', 'Incorrectly predicted nonres',
        'Incorrectly predicted res', 'Correctly predicted res',
        ]
    #     group_counts = ["{0:0.0f}".format(value) for value, count in zip(confusion_matrix.flatten(), counts)]
    group_percentages_and_counts = [f"{round(100*value/count, 2)} %\n {value} of {count}" for value, count in zip(cf_mtx.flatten(), counts)]

    box_labels = [f"{v1}\n {v2}" for v1, v2 in zip(group_names, group_percentages_and_counts)]
    box_labels = np.asarray(box_labels).reshape(2, 2)

    figure, axes = plt.subplots(1, 1, figsize=(11, 9))

    heatmap = sns.heatmap(cf_mtx, xticklabels=get_class_number.keys(), yticklabels=get_class_number.keys(), cmap="PiYG", fmt="", ax=axes, annot=box_labels)

    heatmap.set(
        title='Confusion matrix',
        xlabel='Predicted label',
        ylabel='Actual label'
    )

    return figure

def log_confusion_matrix(epoch, logs):
 
    figure = plot_confusion_matrix()
    cm_image = plot_to_image(figure)

    with file_writer_cm.as_default():
        tf.summary.image("Confusion Matrix", cm_image, step=epoch)
        
        
class MemoryUsageCallback(Callback):
    def on_epoch_begin(self,epoch,logs=None):
        print('Memory usage on epoch begin: {} GB'.format(round(psutil.Process(os.getpid()).memory_info().rss / (1024**3), 3)))

    def on_epoch_end(self,epoch,logs=None):
        print('Memory usage on epoch end:   {} GB'.format(round(psutil.Process(os.getpid()).memory_info().rss / (1024**3), 3)))
        
class ClearMemory(Callback):
    def on_epoch_end(self, epoch, logs=None):        
        collected = gc.collect()
        print(f"Epoch {epoch}: garbage collector collected {collected} objects.")
#         k.clear_session()

In [62]:
# Choose an appropriate optimizer
# SGD, RMSprop, Adam, Adadelta, adagrad
optimizer = Adam(learning_rate=0.001)
B.clear_session()
# Compile the model for multi-class classification
model.compile(
    optimizer=optimizer,
#     run_eagerly=True,
    loss=tf.keras.losses.BinaryCrossentropy(),  # Use CategoricalCrossentropy for multi-class classification
    metrics=[
        'accuracy',
        tf.keras.metrics.TruePositives(name='tp'),
        tf.keras.metrics.TrueNegatives(name='tn'),
        tf.keras.metrics.FalsePositives(name='fp'),
        tf.keras.metrics.FalseNegatives(name='fn'),
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        tf.keras.metrics.AUC(name='auc'),
        tf.keras.metrics.AUC(name='prc', curve='PR')  # Precision-Recall Curve
    ])

# Reducing learning rate for better model fit performance
reduce_learning_rate = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=1, verbose=1, min_delta=0.0005, min_lr=1e-13)
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=checkpoints_and_metadata, histogram_freq=1)
# early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, verbose=1, min_delta=0.005)
log_confusion_matrix = LambdaCallback(on_epoch_end=log_confusion_matrix)
model_checkpoint = ModelCheckpoint(os.path.join(checkpoints_and_metadata, "Epoch_{epoch:02d}-val_accuracy_{val_accuracy:.2f}_checkpoint.h5"), monitor="val_accuracy", verbose=1)


In [ ]:
# Fitting the model on the training data with further evaluation on the validation data during the training process
hist = model.fit(
    train_generator,
    epochs=20,
    steps_per_epoch=int(len_train_images / batch_size),
    callbacks=[reduce_learning_rate, tensorboard_callback, model_checkpoint, MemoryUsageCallback(), ClearMemory()],
    validation_data=([validation_images, validation_numeric], validation_labels),
    shuffle=False,
#     use_multiprocessing=True,
#     workers=8,
    verbose=1)

# save model learning history
with open(f"{checkpoints_and_metadata}/model_history.json", "w") as outfile:
    json.dump(str(hist.history), outfile)

Memory usage on epoch begin: 30.414 GB
Epoch 1/20
1808/2065 [=========================>....] - ETA: 6:45 - loss: 0.3981 - accuracy: 0.8398 - tp: 35518.0000 - tn: 13067.0000 - fp: 5491.0000 - fn: 3780.0000 - precision: 0.8661 - recall: 0.9038 - auc: 0.8794 - prc: 0.9216

In [ ]:
def plot_learning_curves(acc: list, 
                        val_acc: list, 
                        loss: list, 
                        val_loss: list, 
                        precision: list, 
                        val_precision: list, 
                        recall: list, 
                        val_recall: list, 
                        f1: list, 
                        val_f1: list, 
                        prc: list, 
                        val_prc: list):
    plt.figure(figsize=(16, 16))
    plt.subplot(4, 1, 1)
    plt.plot(acc, label='Training Accuracy')
    plt.plot(val_acc, label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.ylabel('Accuracy')
    plt.ylim([min(plt.ylim()),1])
    plt.title('Training and Validation Accuracy')

    plt.subplot(4, 1, 2)
    plt.plot(loss, label='Training Loss')
    plt.plot(val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.ylabel('Cross Entropy')
    plt.ylim([0,1.0])
    plt.title('Training and Validation Loss')
    
    plt.subplot(4, 1, 3)
    plt.plot(f1, label='F1-Score')
    plt.plot(val_f1, label='Validation F1-Score')
    plt.legend(loc='lower right')
    plt.ylabel('F1-Score')
    plt.ylim([min(plt.ylim()),1])
    plt.title('Training and Validation F1-Score')
    
    plt.subplot(4, 1, 4)
    plt.plot(prc, label='AUC-Prec-Recall')
    plt.plot(val_prc, label='Validation AUC-Prec-Recall')
    plt.legend(loc='lower right')
    plt.ylabel('AUC-Prec-Recall')
    plt.ylim([min(plt.ylim()),1])
    plt.title('Training and Validation AUC-Prec-Recall')
    plt.xlabel('epoch')
            
    return plt

In [ ]:
f1_train_scores = [2*p*r/(p+r) if p !=0 or r != 0 else 0 for p, r in zip(hist.history['precision'], hist.history['recall'])]
f1_val_scores = [2*p*r/(p+r) if p !=0 or r != 0 else 0 for p, r in zip(hist.history['val_precision'], hist.history['val_recall'])]

plt = plot_learning_curves(
                        hist.history["accuracy"], 
                        hist.history["val_accuracy"], 
                        hist.history["loss"], 
                        hist.history["val_loss"], 
                        hist.history["precision"], 
                        hist.history["val_precision"], 
                        hist.history["recall"], 
                        hist.history["val_recall"], 
                        f1_train_scores, 
                        f1_val_scores,
                        hist.history["prc"], 
                        hist.history["val_prc"]
                        )

plt.show(block=False)

In [ ]:
print("Garbage collector: collected: ", gc.collect())

In [ ]:
l = os.listdir(checkpoints_and_metadata)

In [ ]:
# zip model metadata
def zipdir(path, ziph):
    # ziph is zipfile handle
    for root, dirs, files in os.walk(checkpoints_and_metadata):
        for file in files:
            ziph.write(os.path.join(root, file), 
                       os.path.relpath(os.path.join(root, file), 
                                       os.path.join(path, '..')))

zip_filename = f"HybridArchitecture_{configuration}_{architecture}_dt{datetime.datetime.now().strftime('%m_%d_%Y_%H_%M_%S')}_checkpoints_and_metadata.zip"
            
with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
    zipdir('/', zipf)

In [ ]:
# save chechpoints and metadata to the bucket
model_name = f"HybridArchitecture_{configuration}_{architecture}_dt{datetime.datetime.now().strftime('%m_%d_%Y_%H_%M_%S')}.h5"
model_path = os.path.join(models_dir, model_name)

model.save(filepath=model_path, overwrite=True)
h5_model_Body=open(model_path, 'rb').read()
cos_client.Object(config["ML_MODELS_BUCKET"], model_name).upload_fileobj(io.BytesIO(h5_model_Body))
print(f'Model uploaded to the Object Cloud Storage ml-saved-models bucket, model name: {model_name}')


zip_Body=open(zip_filename, 'rb').read()
cos_client.Object(config["ML_MODELS_BUCKET"], zip_filename).upload_fileobj(io.BytesIO(zip_Body))

print(f'Checkpoints and metadata were uploaded to the Object Cloud Storage ml-saved-models bucket, model name: {zip_filename}')